In [6]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the input data
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to one-hot encoded format
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu')) 
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(train_images)

# Train the model
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          steps_per_epoch=len(train_images) // 32, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model.save('model/mnist_v5.h5')

Epoch 1/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.2419 - accuracy: 0.9237
Epoch 2/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0869 - accuracy: 0.9732
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0643 - accuracy: 0.9803
Epoch 4/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0548 - accuracy: 0.9827
Epoch 5/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0474 - accuracy: 0.9856
Epoch 6/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0429 - accuracy: 0.9867
Epoch 7/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0378 - accuracy: 0.9888
Epoch 8/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0335 - accuracy: 0.9896
Epoch 9/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0328 - accuracy: 0.9901
Epoch 10/10
313/313 [==============================] - 2s 5ms/st

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import glob
from keras.models import load_model


model = load_model('model/mnist_v5.h5')

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(28, 28), color_mode="grayscale")
    img_array = np.array(img).reshape(-1, 28, 28, 1) 
    img_array = img_array.astype('float32') / 255.0
    return img_array


# Preprocess all the images
image_files = glob.glob('data/*.JPG')  
images = np.vstack([preprocess_image(img_file) for img_file in image_files])

predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Print the filename and corresponding predicted class for each image
for img_file, pred_class in zip(image_files, predicted_classes):
    print(f"File: {img_file}, Predicted Class: {pred_class}")